# SIAMS — End-to-End ML Pipeline (App-Aligned)

**Data source:** Multi-sheet Excel, Google Sheets CSV, local CSV  
**Timezone:** `Africa/Lagos` (WAT, UTC+1)  
**Models:** Decision Tree, Random Forest, Gradient Boosting, XGBoost, Dryness Classifier, t+1 Forecaster  
**Outputs:** `model.joblib`, `expected_features.json`, evaluation metrics, plots, all models

> This notebook provides a clean, self-contained pipeline aligned with Streamlit app and Google Sheet setup. It includes data exploration, time-aware splits, fixed site encoding, clipped predictions, dryness classification, and short-term forecasting.

## 1. Configuration

In [ ]:
# --- Config you can edit ---
# Data sources (set one, leave others empty)
MULTISHEET_XLSX = "SIAMS DataLog.xlsx"   # the xlsx file with multiple sheets (each sheet = one site)
SHEET_CSV_URL   = ""   # Google Sheet CSV export URL
LOCAL_CSV       = ""   # fallback local CSV

# Training settings
KNOWN_SITES     = ["Ikorodu", "Ogun", "Osun", "Unilag"]  # sites for one-hot encoding
TEST_FRACTION   = 0.20  # fraction for test split
RANDOM_STATE    = 42   # for reproducibility

# Dryness classification threshold
DRY_THRESHOLD   = 20.0  # Threshold for dryness classification (moisture %)
                        # 20% = early stress detection (recommended for most crops)
                        # 15% = severe drought stress
                        # 30% = conservative (less sensitive to dry conditions)

# Feature engineering settings
LAG_WINDOWS     = (1, 2, 3, 6)  # Lag periods for soil moisture (hours/readings)
ROLLING_WINDOW  = 6             # Rolling average window size
LIGHT_ADC_MAX   = 4095          # Maximum ADC value for light sensor inversion

# Model hyperparameters (tune for better performance)
RF_N_ESTIMATORS = 300           # Random Forest: number of trees
RF_MAX_DEPTH    = 15            # Random Forest: max tree depth (None = unlimited)
RF_MIN_SAMPLES  = 3             # Random Forest: min samples per leaf

XGB_N_ESTIMATORS = 400          # XGBoost: number of boosting rounds
XGB_LEARNING_RATE = 0.08        # XGBoost: learning rate
XGB_MAX_DEPTH   = 6             # XGBoost: max tree depth

CLF_N_ESTIMATORS = 400          # Classifier: number of trees
CLF_MAX_DEPTH   = 12            # Classifier: max tree depth

# Plant Health Metrics Configuration
VPD_DISEASE_THRESHOLD = 0.6     # VPD threshold for disease risk (kPa)
HEAT_STRESS_TEMP = 35           # Temperature threshold for heat stress (°C)
FROST_RISK_TEMP = 2             # Temperature threshold for frost risk (°C)
WATERLOG_THRESHOLD = 85         # Soil moisture threshold for waterlogging (%)
DRYSPELL_THRESHOLD = 25         # Soil moisture threshold for dry spell (%)
SENSOR_STABILITY_WINDOW = 12    # Rolling window for sensor health check
MOISTURE_PATTERN_WINDOW = 6     # Rolling window for moisture patterns

# Single source of truth for all health metrics
HEALTH_ALL = [
    "vpd_kpa", "dew_point_c", "heat_flag", "frost_flag",
    "waterlog_flag", "dryspell_flag", "disease_risk", "sensor_issue_flag"
]

# Light column headers (add if your data uses different names)
LIGHT_HEADERS_INVERTED = [
    "Relative Light Intensity (ADC Units)",
    "Light Inverted"
]

# Memory management settings
CHUNK_SIZE = 10000          # Process data in chunks for large datasets
MAX_MEMORY_GB = 8           # Maximum memory usage threshold
ENABLE_MEMORY_OPTIMIZATION = True  # Enable memory-saving features

# Imports
import json, warnings, joblib
# Optional psutil import for memory monitoring
try:
    import psutil
    HAVE_PSUTIL = True
except Exception:
    HAVE_PSUTIL = False
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, f1_score, roc_auc_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier, GradientBoostingClassifier

# Optional: XGBoost
try:
    from xgboost import XGBRegressor
    HAVE_XGB = True
except Exception:
    HAVE_XGB = False
    warnings.warn("XGBoost not available; skipping XGBoost.")

## 2. Data Loading

In [ ]:
# Data Loading Functions
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def check_memory_usage():
    """Monitor memory usage and warn if approaching limits."""
    if not (ENABLE_MEMORY_OPTIMIZATION and HAVE_PSUTIL):
        return False
    
    try:
        memory_info = psutil.virtual_memory()
        memory_gb = memory_info.used / (1024**3)
        memory_percent = memory_info.percent
        
        if memory_gb > MAX_MEMORY_GB * 0.8:
            logger.warning(f"High memory usage: {memory_gb:.1f}GB / {MAX_MEMORY_GB}GB limit ({memory_percent:.1f}%)")
            return True
        elif memory_percent > 85:
            logger.warning(f"System memory usage high: {memory_percent:.1f}% ({memory_gb:.1f}GB)")
            return True
        
        return False
    except Exception as e:
        logger.warning(f"Memory monitoring failed: {e}")
        return False

def load_multisheet_xlsx(path: str) -> pd.DataFrame:
    """Load multi-sheet Excel file, adding site_id from sheet names."""
    path = Path(path)
    if not path.exists():
        raise FileNotFoundError(f"Excel file not found: {path}")
    
    # Check memory before large operation
    check_memory_usage()
    
    try:
        book = pd.read_excel(path, sheet_name=None)
    except Exception as e:
        raise ValueError(f"Failed to read Excel file {path}: {e}")
    
    frames = []
    for sheet_name, df in book.items():
        if df.empty:
            logger.warning(f"Empty sheet '{sheet_name}' skipped")
            continue
        df = df.copy()
        df["site_id"] = sheet_name
        frames.append(df)
        logger.info(f"Loaded sheet '{sheet_name}' with {len(df)} rows")
    
    if not frames:
        raise ValueError("No non-empty sheets found in Excel file.")
    
    result = pd.concat(frames, ignore_index=True)
    logger.info(f"Combined {len(frames)} sheets into {len(result)} total rows")
    
    # Check memory after operation
    check_memory_usage()
    return result

def load_google_sheet_csv(url: str) -> pd.DataFrame:
    """Load CSV from Google Sheets URL."""
    if not url:
        raise ValueError("SHEET_CSV_URL is empty.")
    
    logger.info(f"Loading data from Google Sheets URL")
    check_memory_usage()
    
    try:
        return pd.read_csv(url)
    except Exception as e:
        logger.warning(f"Standard CSV read failed: {e}. Trying with error handling...")
        try:
            return pd.read_csv(url, engine="python", on_bad_lines="skip")
        except Exception as e2:
            raise ValueError(f"Failed to load Google Sheets CSV: {e2}")

def load_local_csv(path: str) -> pd.DataFrame:
    """Load local CSV file."""
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"Local CSV not found: {p}")
    
    check_memory_usage()
    try:
        result = pd.read_csv(p)
        logger.info(f"Loaded local CSV with {len(result)} rows")
        return result
    except Exception as e:
        raise ValueError(f"Failed to read CSV file {p}: {e}")

def validate_dataframe(df: pd.DataFrame, min_rows: int = 10) -> None:
    """Validate basic dataframe properties."""
    if df.empty:
        raise ValueError("Dataframe is empty")
    if len(df) < min_rows:
        raise ValueError(f"Dataframe has only {len(df)} rows, minimum {min_rows} required")
    
    # Check for completely empty columns
    empty_cols = df.columns[df.isnull().all()].tolist()
    if empty_cols:
        logger.warning(f"Completely empty columns found: {empty_cols}")

def unify_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Standardize column names using aliases."""
    if df.empty:
        raise ValueError("Cannot unify columns of empty dataframe")
    
    df = df.copy()
    original_cols = df.columns.tolist()
    df.columns = [str(c).strip() for c in df.columns]
    
    alias = {
        "Date": "timestamp",
        "Soil Moisture %": "soil_moisture_pct",
        "Soil Moisture (%)": "soil_moisture_pct",
        "Humidity %": "humidity_pct",
        "Humidity (%)": "humidity_pct",
        "Temperature °C": "temperature_c",
        "Temperature (°C)": "temperature_c",
        "Light Intensity": "light_adc",
        "Site": "site_id",
        "site": "site_id",
        "location": "site_id",
    }
    
    renamed_count = 0
    for old, new in alias.items():
        if old in df.columns and new not in df.columns:
            df[new] = df[old]
            renamed_count += 1
    
    logger.info(f"Renamed {renamed_count} columns using aliases")
    
    # Handle timestamps with better error handling
    timestamp_cols = ["timestamp", "Date"]
    timestamp_col = None
    
    for col in timestamp_cols:
        if col in df.columns:
            timestamp_col = col
            break
    
    if timestamp_col:
        try:
            ts = pd.to_datetime(df[timestamp_col], errors="coerce", utc=True)
            if ts.isna().mean() > 0.5:
                logger.warning("Many timestamp parsing failures, trying without UTC")
                ts = pd.to_datetime(df[timestamp_col], errors="coerce")
            
            failed_parses = ts.isna().sum()
            if failed_parses > 0:
                logger.warning(f"{failed_parses} timestamps failed to parse")
            
            df["timestamp"] = ts
        except Exception as e:
            logger.error(f"Timestamp parsing failed: {e}")
            raise
    else:
        logger.warning("No timestamp column found in data")
    
    return df

def choose_light_column(df: pd.DataFrame) -> str:
    """Select the appropriate light column, inverting if needed, and standardize to 'light_adc' with normalized version."""
    if df.empty:
        raise ValueError("Cannot choose light column from empty dataframe")
    
    # Check for pre-inverted columns first
    for c in LIGHT_HEADERS_INVERTED:
        if c in df.columns:
            non_null_count = df[c].notna().sum()
            if non_null_count > 0:
                df["light_adc"] = df[c]  # Rename pre-inverted column to 'light_adc'
                logger.info(f"Using pre-inverted light column '{c}' with {non_null_count} valid values")
                
                # Always provide a normalized light feature in [0,1] with clamping
                df["light_norm"] = pd.to_numeric(df["light_adc"], errors="coerce") / float(LIGHT_ADC_MAX)
                df["light_norm"] = df["light_norm"].clip(lower=0.0, upper=1.0)  # Clamp to [0,1]
                
                # Log any clamped values
                original_norm = pd.to_numeric(df["light_adc"], errors="coerce") / float(LIGHT_ADC_MAX)
                clamped_low = (original_norm < 0.0).sum()
                clamped_high = (original_norm > 1.0).sum()
                if clamped_low > 0 or clamped_high > 0:
                    logger.warning(f"Clamped light values - Low: {clamped_low}, High: {clamped_high}")
                
                logger.info(f"Created normalized light feature 'light_norm' (0-1 scale, clamped)")
                
                return "light_adc"
    
    # Check for raw ADC column that needs inversion
    if "light_adc" in df.columns:
        try:
            original_values = pd.to_numeric(df["light_adc"], errors="coerce")
            non_null_count = original_values.notna().sum()
            
            if non_null_count == 0:
                raise ValueError("light_adc column contains no valid numeric data")
            
            # Validate ADC range
            min_val, max_val = original_values.min(), original_values.max()
            if max_val > LIGHT_ADC_MAX:
                logger.warning(f"Light ADC values exceed expected max {LIGHT_ADC_MAX}: max={max_val}")
            
            df["light_adc"] = LIGHT_ADC_MAX - original_values  # Invert raw ADC
            logger.info(f"Inverted raw light_adc column (range: {min_val}-{max_val})")
            
            # Always provide a normalized light feature in [0,1] with clamping
            df["light_norm"] = pd.to_numeric(df["light_adc"], errors="coerce") / float(LIGHT_ADC_MAX)
            df["light_norm"] = df["light_norm"].clip(lower=0.0, upper=1.0)  # Clamp to [0,1]
            
            # Log any clamped values  
            original_norm = pd.to_numeric(df["light_adc"], errors="coerce") / float(LIGHT_ADC_MAX)
            clamped_low = (original_norm < 0.0).sum()
            clamped_high = (original_norm > 1.0).sum()
            if clamped_low > 0 or clamped_high > 0:
                logger.warning(f"Clamped light values - Low: {clamped_low}, High: {clamped_high}")
            
            logger.info(f"Created normalized light feature 'light_norm' (0-1 scale, clamped)")
            
            return "light_adc"
        except Exception as e:
            raise ValueError(f"Failed to process light_adc column: {e}")
    
    available_cols = [c for c in df.columns if 'light' in c.lower()]
    raise ValueError(f"No light column found. Expected one of {LIGHT_HEADERS_INVERTED} "
                     f"or raw 'light_adc'. Available light-related columns: {available_cols}")

# Load data based on config with comprehensive error handling
try:
    if MULTISHEET_XLSX:
        logger.info(f"Loading multi-sheet Excel: {MULTISHEET_XLSX}")
        raw = load_multisheet_xlsx(MULTISHEET_XLSX)
    elif SHEET_CSV_URL:
        logger.info("Loading from Google Sheets CSV URL")
        raw = load_google_sheet_csv(SHEET_CSV_URL)
    else:
        logger.info(f"Loading local CSV: {LOCAL_CSV}")
        raw = load_local_csv(LOCAL_CSV)
    
    # Validate loaded data
    validate_dataframe(raw, min_rows=50)  # Require at least 50 rows for meaningful ML
    
    # Process columns
    raw = unify_columns(raw)
    light_col = choose_light_column(raw)
    
    # Check for required columns
    required = ["timestamp", "soil_moisture_pct", "temperature_c", "humidity_pct"]
    missing = [c for c in required if c not in raw.columns]
    
    if missing:
        logger.error(f"Missing required columns: {missing}")
        available = list(raw.columns)
        logger.info(f"Available columns: {available}")
        raise ValueError(f"Missing required columns: {missing}")
    
    # Validate data ranges
    for col in ["soil_moisture_pct", "humidity_pct"]:
        if col in raw.columns:
            valid_data = pd.to_numeric(raw[col], errors="coerce")
            out_of_range = ((valid_data < 0) | (valid_data > 100)).sum()
            if out_of_range > 0:
                logger.warning(f"{col}: {out_of_range} values outside 0-100% range")
    
    logger.info(f"Successfully loaded and validated {len(raw)} rows")
    print(f"Data loaded successfully: {len(raw)} rows, {len(raw.columns)} columns")
    raw.tail(3)

except Exception as e:
    logger.error(f"Data loading failed: {e}")
    print(f"Error loading data: {e}")
    raise

In [ ]:
# Save cleaned data as CSV
raw.to_csv("siams_cleaned.csv", index=False)
print("Saved cleaned data to: siams_cleaned.csv")

In [ ]:
# Summary per Site
summary = raw.groupby("site_id")["timestamp"].agg(n_records="count", start="min", end="max").reset_index()
summary

In [ ]:
# Save summary as CSV
summary.to_csv("siams_summary.csv", index=False)
print("Saved summary to: siams_summary.csv")

In [ ]:
# EDA Plots (Matplotlib only)
import matplotlib.pyplot as plt

metrics = [
    ("temperature_c","Temperature (°C)"),
    ("humidity_pct","Humidity (%)"),
    ("soil_moisture_pct","Soil Moisture (%)"),
    ("light_norm","Light Intensity (Normalized 0-1)")  # Updated to use normalized light
]

plot_dir = Path("plots")
plot_dir.mkdir(exist_ok=True, parents=True)

sites = sorted(raw["site_id"].dropna().unique())

for col, label in metrics:
    if col not in raw.columns:
        continue
    plt.figure()
    data = [raw.loc[raw["site_id"]==s, col].dropna() for s in sites]
    plt.boxplot(data, tick_labels=sites, showfliers=False)
    plt.title(f"{label} by Site — Boxplot")
    plt.xlabel("Site"); plt.ylabel(label)
    plt.tight_layout()
    plt.savefig(plot_dir / f"boxplot_{col}.png", dpi=160)
    plt.show()

for col, label in metrics:
    if col not in raw.columns:
        continue
    for s in sites:
        subset = raw.loc[raw["site_id"]==s, col].dropna()
        if subset.empty:
            continue
        plt.figure()
        plt.hist(subset, bins=30)
        plt.title(f"{label} — Histogram ({s})")
        plt.xlabel(label); plt.ylabel("Count")
        plt.tight_layout()
        plt.savefig(plot_dir / f"hist_{col}_{s}.png", dpi=160)
        plt.show()

print("Saved plots to:", plot_dir)

## 3. Feature Engineering

In [ ]:
# Feature Engineering Functions
def add_calendar_features(df: pd.DataFrame) -> pd.DataFrame:
    """Add time-based features: hour, dayofweek, month (Africa/Lagos timezone).
    
    Handles both timezone-aware and timezone-naive timestamps safely.
    """
    df = df.copy()
    ts = pd.to_datetime(df["timestamp"], errors="coerce")
    
    # If ts is tz-naive, localize to UTC first, then convert
    if ts.dt.tz is None:
        ts = ts.dt.tz_localize("UTC", nonexistent="NaT", ambiguous="NaT")
    
    ts_local = ts.dt.tz_convert("Africa/Lagos")
    df["hour"] = ts_local.dt.hour
    df["dayofweek"] = ts_local.dt.dayofweek
    df["month"] = ts_local.dt.month
    return df

def add_lags_and_rolls(df: pd.DataFrame, lags=(1,2,3), roll_window=3) -> pd.DataFrame:
    """Add lagged and rolling features for soil moisture."""
    if "site_id" in df.columns:
        df = df.sort_values(["site_id","timestamp"]).copy()
        def _apply(g):
            for L in lags:
                g[f"soil_moisture_lag{L}"] = g["soil_moisture_pct"].shift(L)
            if roll_window and roll_window > 1:
                past = g["soil_moisture_pct"].shift(1)
                g[f"soil_moisture_roll{roll_window}"] = past.rolling(roll_window, min_periods=1).mean()
            return g
        df = df.groupby("site_id", group_keys=False).apply(_apply)
    else:
        df = df.sort_values(["timestamp"]).copy()
        for L in lags:
            df[f"soil_moisture_lag{L}"] = df["soil_moisture_pct"].shift(L)
        if roll_window and roll_window > 1:
            past = df["soil_moisture_pct"].shift(1)
            df[f"soil_moisture_roll{roll_window}"] = past.rolling(roll_window, min_periods=1).mean()
    return df

def add_plant_health_metrics(df: pd.DataFrame) -> pd.DataFrame:
    """Add comprehensive plant health and environmental stress metrics."""
    logger.info("Calculating plant health metrics...")
    df = df.copy()
    
    # Convert to numeric for calculations
    T = pd.to_numeric(df.get("temperature_c"), errors="coerce")
    RH = pd.to_numeric(df.get("humidity_pct"), errors="coerce")
    SM = pd.to_numeric(df.get("soil_moisture_pct"), errors="coerce")
    
    # Validate input ranges and log warnings
    invalid_temp = ((T < -50) | (T > 70)).sum()
    invalid_humidity = ((RH < 0) | (RH > 100)).sum()
    invalid_moisture = ((SM < 0) | (SM > 100)).sum()
    
    if invalid_temp > 0:
        logger.warning(f"{invalid_temp} temperature readings outside realistic range (-50°C to 70°C)")
    if invalid_humidity > 0:
        logger.warning(f"{invalid_humidity} humidity readings outside 0-100% range")
    if invalid_moisture > 0:
        logger.warning(f"{invalid_moisture} soil moisture readings outside 0-100% range")
    
    # Check for sensor data availability
    temp_available = T.notna().sum()
    humidity_available = RH.notna().sum()
    moisture_available = SM.notna().sum()
    
    if temp_available == 0:
        logger.error("No valid temperature data available for health metrics")
        raise ValueError("Temperature data required for health metrics calculation")
    if humidity_available == 0:
        logger.error("No valid humidity data available for health metrics")
        raise ValueError("Humidity data required for health metrics calculation")
    if moisture_available == 0:
        logger.error("No valid soil moisture data available for health metrics")
        raise ValueError("Soil moisture data required for health metrics calculation")
    
    logger.info(f"Health metrics input validation: {temp_available} temp, {humidity_available} humidity, {moisture_available} moisture readings")
    
    # Check memory usage before intensive calculations
    if check_memory_usage():
        logger.info("High memory usage detected, optimizing calculations...")
    
    # Vapor Pressure Deficit (VPD) in kPa - critical for plant stress assessment
    try:
        es = 0.6108 * np.exp((17.27 * T) / (T + 237.3))  # Saturation vapor pressure
        ea = es * (RH / 100.0)  # Actual vapor pressure
        df["vpd_kpa"] = (es - ea).clip(lower=0)
        
        # Validate VPD range (typically 0-6 kPa)
        extreme_vpd = (df["vpd_kpa"] > 6.0).sum()
        if extreme_vpd > 0:
            logger.warning(f"{extreme_vpd} VPD values exceed 6.0 kPa (extreme conditions)")
    except Exception as e:
        logger.error(f"VPD calculation failed: {e}")
        df["vpd_kpa"] = np.nan
    
    # Dew point calculation (°C) - important for disease risk
    try:
        a, b = 17.27, 237.3
        alpha = (a * T) / (b + T) + np.log((RH / 100.0).clip(lower=1e-6))
        df["dew_point_c"] = (b * alpha) / (a - alpha)
        
        # Validate dew point (should be <= temperature)
        invalid_dewpoint = (df["dew_point_c"] > T).sum()
        if invalid_dewpoint > 0:
            logger.warning(f"{invalid_dewpoint} dew point values exceed temperature (calculation error)")
    except Exception as e:
        logger.error(f"Dew point calculation failed: {e}")
        df["dew_point_c"] = np.nan
    
    # Environmental stress flags with validation
    df["heat_flag"] = (T >= HEAT_STRESS_TEMP).astype(int)   # Heat stress indicator
    df["frost_flag"] = (T <= FROST_RISK_TEMP).astype(int)   # Frost risk indicator
    
    heat_events = df["heat_flag"].sum()
    frost_events = df["frost_flag"].sum()
    if heat_events > 0:
        logger.info(f"Detected {heat_events} heat stress events (≥{HEAT_STRESS_TEMP}°C)")
    if frost_events > 0:
        logger.info(f"Detected {frost_events} frost risk events (≤{FROST_RISK_TEMP}°C)")
    
    # Soil moisture pattern analysis (rolling averages per site)
    try:
        if "site_id" in df.columns:
            df["moist_roll6"] = df.groupby("site_id")["soil_moisture_pct"].transform(
                lambda s: pd.to_numeric(s, errors="coerce").rolling(MOISTURE_PATTERN_WINDOW, min_periods=3).mean()
            )
        else:
            df["moist_roll6"] = pd.to_numeric(df["soil_moisture_pct"], errors="coerce").rolling(MOISTURE_PATTERN_WINDOW, min_periods=3).mean()
        
        df["waterlog_flag"] = (df["moist_roll6"] >= WATERLOG_THRESHOLD).astype(int)  # Waterlogging risk
        df["dryspell_flag"] = (df["moist_roll6"] <= DRYSPELL_THRESHOLD).astype(int)  # Drought stress
        
        waterlog_events = df["waterlog_flag"].sum()
        dryspell_events = df["dryspell_flag"].sum()
        if waterlog_events > 0:
            logger.info(f"Detected {waterlog_events} waterlogging risk events (≥{WATERLOG_THRESHOLD}%)")
        if dryspell_events > 0:
            logger.info(f"Detected {dryspell_events} dry spell events (≤{DRYSPELL_THRESHOLD}%)")
    except Exception as e:
        logger.error(f"Moisture pattern analysis failed: {e}")
        df["moist_roll6"] = np.nan
        df["waterlog_flag"] = 0
        df["dryspell_flag"] = 0
    
    # Disease risk assessment (0-100 scale) with validation
    try:
        # High risk conditions: high humidity (>85%), optimal temp (18-28°C), low VPD (<0.6 kPa)
        temp_optimal = ((T >= 18) & (T <= 28)).astype(int)
        vpd_conducive = (df["vpd_kpa"] <= VPD_DISEASE_THRESHOLD).astype(int)
        humidity_high = (RH >= 85).astype(int)
        
        # Weighted disease risk score
        df["disease_risk"] = (
            (0.5 * humidity_high + 0.3 * temp_optimal + 0.2 * vpd_conducive) / (0.5 + 0.3 + 0.2) * 100
        ).round(1)
        
        high_risk_periods = (df["disease_risk"] > 75).sum()
        if high_risk_periods > 0:
            logger.info(f"Detected {high_risk_periods} high disease risk periods (>75% score)")
    except Exception as e:
        logger.error(f"Disease risk calculation failed: {e}")
        df["disease_risk"] = 0
    
    # Sensor health monitoring - detect "stuck" sensors (low variability)
    sensor_columns = ["soil_moisture_pct", "temperature_c", "humidity_pct", "light_adc"]
    sensor_issues_detected = 0
    
    try:
        if "site_id" in df.columns:
            # Per-site sensor health analysis
            for col in sensor_columns:
                if col in df.columns:
                    rolling_std = df.groupby("site_id")[col].transform(
                        lambda s: pd.to_numeric(s, errors="coerce").rolling(SENSOR_STABILITY_WINDOW, min_periods=6).std()
                    )
                    df[f"{col}_stuck"] = (rolling_std.fillna(0) == 0).astype(int)
                    stuck_count = df[f"{col}_stuck"].sum()
                    if stuck_count > 0:
                        sensor_issues_detected += stuck_count
                        logger.warning(f"Sensor {col}: {stuck_count} potential stuck readings detected")
        else:
            # Global sensor health analysis
            for col in sensor_columns:
                if col in df.columns:
                    rolling_std = pd.to_numeric(df[col], errors="coerce").rolling(SENSOR_STABILITY_WINDOW, min_periods=6).std()
                    df[f"{col}_stuck"] = (rolling_std.fillna(0) == 0).astype(int)
                    stuck_count = df[f"{col}_stuck"].sum()
                    if stuck_count > 0:
                        sensor_issues_detected += stuck_count
                        logger.warning(f"Sensor {col}: {stuck_count} potential stuck readings detected")
        
        # Overall sensor issue flag
        stuck_columns = [f"{c}_stuck" for c in sensor_columns if f"{c}_stuck" in df.columns]
        if stuck_columns:
            df["sensor_issue_flag"] = df[stuck_columns].any(axis=1).astype(int)
            total_sensor_issues = df["sensor_issue_flag"].sum()
            if total_sensor_issues > 0:
                logger.warning(f"Total sensor issues detected: {total_sensor_issues} readings flagged")
        else:
            df["sensor_issue_flag"] = 0
    except Exception as e:
        logger.error(f"Sensor health monitoring failed: {e}")
        df["sensor_issue_flag"] = 0
    
    # Final validation and summary
    health_metrics_created = ["vpd_kpa", "dew_point_c", "heat_flag", "frost_flag", 
                             "waterlog_flag", "dryspell_flag", "disease_risk", "sensor_issue_flag"]
    
    successful_metrics = [col for col in health_metrics_created if col in df.columns and df[col].notna().any()]
    failed_metrics = [col for col in health_metrics_created if col not in df.columns or df[col].isna().all()]
    
    if failed_metrics:
        logger.warning(f"Failed to create health metrics: {failed_metrics}")
    
    logger.info(f"Plant health metrics calculation completed: {len(successful_metrics)}/{len(health_metrics_created)} metrics successful")
    return df

def one_hot_encode_sites(df: pd.DataFrame, known_sites) -> pd.DataFrame:
    """One-hot encode site_id with fixed order."""
    df = df.copy()
    if "site_id" not in df.columns:
        df["site_id"] = known_sites[0]  # fallback
    dummies = pd.get_dummies(df["site_id"], prefix="site_id")
    desired = [f"site_id_{s}" for s in known_sites]
    for col in desired:
        if col not in dummies.columns:
            dummies[col] = 0
    dummies = dummies.reindex(columns=desired, fill_value=0)
    return pd.concat([df.drop(columns=["site_id"], errors="ignore"), dummies], axis=1)

# Apply comprehensive feature engineering
logger.info("Starting comprehensive feature engineering...")
features = raw.copy()
features = add_calendar_features(features)
features = add_lags_and_rolls(features, lags=LAG_WINDOWS, roll_window=ROLLING_WINDOW)
features = add_plant_health_metrics(features)  # Add health metrics
features = one_hot_encode_sites(features, KNOWN_SITES)

# Define comprehensive feature columns including health metrics
site_cols = [f"site_id_{s}" for s in KNOWN_SITES]
HEALTH_COLS_TRAIN = ["vpd_kpa", "dew_point_c", "heat_flag", "frost_flag", "disease_risk"]  # safe (no target leakage)
HEALTH_EXPORT_COLS = [
    "timestamp", "site_id", "vpd_kpa", "dew_point_c", "heat_flag", "frost_flag",
    "waterlog_flag", "dryspell_flag", "disease_risk", "sensor_issue_flag"
]  # for app display

feat_cols = [
    # Core environmental features
    "temperature_c", "humidity_pct", "light_norm",   # <- Changed from light_adc to light_norm
    # Temporal features
    "hour", "dayofweek", "month",
    # Soil moisture patterns
    *[f"soil_moisture_lag{L}" for L in LAG_WINDOWS],
    f"soil_moisture_roll{ROLLING_WINDOW}",
    # Plant health and stress indicators (training-safe only)
    *HEALTH_COLS_TRAIN,
    # Site identification
    *site_cols
]

# Ensure all features exist
for c in feat_cols:
    if c not in features.columns:
        features[c] = np.nan

# Prepare X and y but keep original indices for splitting
X_full = features[feat_cols].astype(float)
y_full = pd.to_numeric(features["soil_moisture_pct"], errors="coerce")

# Apply mask to identify valid rows
mask = X_full.notna().all(axis=1) & y_full.notna()

# Filter to get clean data, preserving original indices
X = X_full[mask]
y = y_full[mask]

print("Final feature order used for training/inference:")
print(feat_cols)
print(f"Total features: {len(feat_cols)} (including {len(HEALTH_COLS_TRAIN)} health metrics)")
print("Rows available:", len(X))
X.tail(3)

In [ ]:
# Save features as CSV
features.to_csv("siams_features.csv", index=False)
print("Saved features to: siams_features.csv")

# Export health metrics for the Streamlit app
# Note: HEALTH_EXPORT_COLS is already defined earlier near feature engineering

def reconstruct_site_id(df, known_sites=KNOWN_SITES):
    """Reconstruct site_id from one-hot encoded columns."""
    scols = [f"site_id_{s}" for s in known_sites]
    present = [c for c in scols if c in df.columns]
    if not present:
        return pd.Series("unknown", index=df.index)
    return df[present].idxmax(axis=1).str.replace("site_id_", "", regex=False)

# Reconstruct site_id from one-hot columns before export (since it's dropped during one-hot encoding)
if "site_id" not in features.columns:
    features["site_id"] = reconstruct_site_id(features)

# Ensure all columns exist before saving
available_health_cols = [col for col in HEALTH_EXPORT_COLS if col in features.columns]
missing_cols = set(HEALTH_EXPORT_COLS) - set(available_health_cols)
if missing_cols:
    logger.warning(f"Missing health export columns: {missing_cols}")

health_export = features[available_health_cols].copy()
health_export.to_csv("siams_health_metrics.csv", index=False)
logger.info(f"Exported health metrics with {len(available_health_cols)} columns to siams_health_metrics.csv")

# Display health metrics summary
print("\nPlant Health Metrics Summary:")
health_summary_cols = [col for col in HEALTH_ALL if col in features.columns]
if health_summary_cols:
    health_summary = features[health_summary_cols].describe()
    print(health_summary)
else:
    print("No health metrics available for summary")

## 4. Model Training and Evaluation

In [ ]:
# Model Training and Evaluation

def validate_model_inputs(X: pd.DataFrame, y: pd.Series, min_samples: int = 100) -> None:
    """Validate inputs for model training."""
    if X.empty or y.empty:
        raise ValueError("Empty training data provided")
    
    if len(X) != len(y):
        raise ValueError(f"Feature matrix ({len(X)}) and target ({len(y)}) length mismatch")
    
    if len(X) < min_samples:
        raise ValueError(f"Insufficient data: {len(X)} samples, minimum {min_samples} required")
    
    # Check for infinite or NaN values
    if X.isnull().any().any():
        null_cols = X.columns[X.isnull().any()].tolist()
        raise ValueError(f"NaN values found in features: {null_cols}")
    
    if y.isnull().any():
        raise ValueError("NaN values found in target variable")
    
    if np.isinf(X.values).any():
        raise ValueError("Infinite values found in feature matrix")
    
    if np.isinf(y.values).any():
        raise ValueError("Infinite values found in target variable")
    
    logger.info(f"Model input validation passed: {len(X)} samples, {len(X.columns)} features")

def infer_site_label(df_feat: pd.DataFrame) -> pd.Series:
    """Infer site labels from one-hot encoded columns."""
    scols = [f"site_id_{s}" for s in KNOWN_SITES]
    
    if not all(c in df_feat.columns for c in scols):
        missing_sites = [c for c in scols if c not in df_feat.columns]
        logger.warning(f"Missing site columns: {missing_sites}")
        return pd.Series(["unknown"] * len(df_feat), index=df_feat.index)
    
    # Check for rows with no site assigned (all zeros)
    site_matrix = df_feat[scols]
    row_sums = site_matrix.sum(axis=1)
    unassigned = (row_sums == 0).sum()
    
    if unassigned > 0:
        logger.warning(f"{unassigned} rows have no site assignment")
    
    # Handle multiple site assignments (should not happen with proper one-hot encoding)
    multiple_sites = (row_sums > 1).sum()
    if multiple_sites > 0:
        logger.warning(f"{multiple_sites} rows have multiple site assignments")
    
    lab = site_matrix.idxmax(axis=1).str.replace("site_id_", "", regex=False)
    return lab

def time_aware_split_indices(df_feat: pd.DataFrame, test_fraction: float = TEST_FRACTION) -> tuple:
    """Perform time-aware train/test split per site."""
    if not 0 < test_fraction < 1:
        raise ValueError(f"test_fraction must be between 0 and 1, got {test_fraction}")
    
    lab = infer_site_label(df_feat)
    
    # Get timestamps for the filtered dataframe indices
    ts = pd.to_datetime(raw.loc[df_feat.index, "timestamp"], errors="coerce")
    
    if ts.isnull().any():
        null_timestamps = ts.isnull().sum()
        logger.warning(f"{null_timestamps} rows have invalid timestamps")
        # Remove rows with invalid timestamps
        valid_mask = ts.notnull()
        df_feat = df_feat[valid_mask]
        lab = lab[valid_mask]
        ts = ts[valid_mask]
    
    train_idx, test_idx = [], []
    site_stats = {}
    
    for site in lab.unique():
        if site == "unknown":
            continue
            
        site_mask = (lab == site)
        idx = df_feat.index[site_mask]
        site_timestamps = ts.loc[idx]
        
        # TZ-safe time sort with fallback for older pandas versions
        ts_utc = pd.to_datetime(site_timestamps, utc=True)
        try:
            ints = ts_utc.view("int64").values
        except Exception:
            ints = ts_utc.astype("int64").values
        order = np.argsort(ints)
        sorted_indices = idx[order]
        
        n = len(sorted_indices)
        if n == 0:
            continue
            
        k = int(round((1 - test_fraction) * n))
        
        # Ensure at least one sample in each split
        k = max(1, min(k, n - 1))
        
        site_train = sorted_indices[:k]
        site_test = sorted_indices[k:]
        
        train_idx.extend(site_train)
        test_idx.extend(site_test)
        
        site_stats[site] = {
            'total': n,
            'train': len(site_train),
            'test': len(site_test),
            'train_end': site_timestamps.loc[site_train].max(),
            'test_start': site_timestamps.loc[site_test].min()
        }
    
    # Log split statistics
    total_train, total_test = len(train_idx), len(test_idx)
    actual_test_fraction = total_test / (total_train + total_test)
    
    logger.info(f"Time-aware split: {total_train} train, {total_test} test "
                f"({actual_test_fraction:.1%} test fraction)")
    
    for site, stats in site_stats.items():
        logger.info(f"Site {site}: {stats['train']} train, {stats['test']} test")
    
    return np.array(train_idx), np.array(test_idx)

# Split data with validation - use filtered features dataframe
try:
    # Use the filtered features (those with valid data) for the split
    features_filtered = features.loc[X.index]
    train_idx, test_idx = time_aware_split_indices(features_filtered)
    
    # Validate splits
    if len(train_idx) == 0 or len(test_idx) == 0:
        raise ValueError("Empty train or test split")
    
    # Check for overlap (should never happen)
    overlap = set(train_idx) & set(test_idx)
    if overlap:
        raise ValueError(f"Train/test overlap detected: {len(overlap)} samples")
    
    # Ensure all indices exist in our filtered data
    train_idx = [idx for idx in train_idx if idx in X.index]
    test_idx = [idx for idx in test_idx if idx in X.index]
    
    # Final validation
    if len(train_idx) < 50:
        raise ValueError(f"Insufficient training samples after filtering: {len(train_idx)}")
    if len(test_idx) < 10:
        raise ValueError(f"Insufficient test samples after filtering: {len(test_idx)}")
    
    logger.info(f"Final split - Train: {len(train_idx)}, Test: {len(test_idx)}")

except Exception as e:
    logger.error(f"Train/test split failed: {e}")
    raise

# Define models with better configuration
models = {
    "DecisionTree": DecisionTreeRegressor(
        random_state=RANDOM_STATE,
        min_samples_leaf=2,  # Prevent overfitting
        min_samples_split=5   # Prevent overfitting
    ),
    "RandomForest": RandomForestRegressor(
        n_estimators=RF_N_ESTIMATORS, 
        max_depth=RF_MAX_DEPTH, 
        min_samples_leaf=RF_MIN_SAMPLES,
        random_state=RANDOM_STATE, 
        n_jobs=-1,
        oob_score=True  # Out-of-bag scoring for additional validation
    ),
    "GradientBoosting": GradientBoostingRegressor(
        random_state=RANDOM_STATE,
        validation_fraction=0.1,  # Use for early stopping
        n_iter_no_change=10       # Early stopping patience
    ),
}

if HAVE_XGB:
    models["XGBoost"] = XGBRegressor(
        n_estimators=XGB_N_ESTIMATORS, 
        learning_rate=XGB_LEARNING_RATE, 
        max_depth=XGB_MAX_DEPTH,
        subsample=0.9, 
        colsample_bytree=0.9, 
        random_state=RANDOM_STATE,
        n_jobs=-1, 
        reg_lambda=1.0, 
        tree_method="hist",
        eval_metric="rmse"         # Explicit evaluation metric
    )

def evaluate(model, Xtr: pd.DataFrame, ytr: pd.Series, Xte: pd.DataFrame, yte: pd.Series) -> tuple:
    """Train model and compute metrics, clipping predictions."""
    # Validate inputs
    validate_model_inputs(Xtr, ytr)
    validate_model_inputs(Xte, yte, min_samples=10)  # Less strict for test set
    
    # Detect XGBoost using proper type checking
    try:
        import xgboost as xgb
        is_xgb = isinstance(model, xgb.XGBRegressor)
    except Exception:
        is_xgb = False

    if is_xgb:
        # XGBoost early stopping with validation split
        val_size = max(10, int(0.1 * len(Xtr)))
        model.fit(
            Xtr.iloc[:-val_size], ytr.iloc[:-val_size],
            eval_set=[(Xtr.iloc[-val_size:], ytr.iloc[-val_size:])],
            verbose=False
        )
    else:
        model.fit(Xtr, ytr)
    
    # Make predictions and clip to realistic range
    pred = model.predict(Xte)
    pred_clipped = np.clip(pred, 0, 100)
    
    # Log clipping statistics
    clipped_count = (pred != pred_clipped).sum()
    if clipped_count > 0:
        logger.info(f"Clipped {clipped_count}/{len(pred)} predictions to 0-100% range")
    
    # Calculate metrics - Fix for older scikit-learn versions
    mse = mean_squared_error(yte, pred_clipped)
    rmse = np.sqrt(mse)  # Calculate RMSE manually
    mae = mean_absolute_error(yte, pred_clipped)
    r2 = r2_score(yte, pred_clipped)
    
    return rmse, mae, r2, pred_clipped

# Train and evaluate models with comprehensive error handling
rows = []
best = None
model_errors = {}

for name, mdl in models.items():
    try:
        logger.info(f"Training {name} model...")
        rmse, mae, r2, _ = evaluate(mdl, X.loc[train_idx], y.loc[train_idx], 
                                    X.loc[test_idx], y.loc[test_idx])
        
        rows.append({"Model": name, "RMSE": rmse, "MAE": mae, "R2": r2})
        
        # Track best model
        if best is None or r2 > best["R2"] or (r2 == best["R2"] and rmse < best["RMSE"]):
            best = {"Name": name, "Model": mdl, "RMSE": rmse, "MAE": mae, "R2": r2}
        
        logger.info(f"{name}: RMSE={rmse:.3f}, MAE={mae:.3f}, R²={r2:.3f}")
        
    except Exception as e:
        error_msg = f"Failed to train {name}: {e}"
        logger.error(error_msg)
        model_errors[name] = str(e)
        # Continue with other models

# Enhanced error recovery with fallback baseline model
if not rows:
    logger.warning("All advanced models failed, attempting baseline model as fallback...")
    try:
        # Fallback baseline model with minimal complexity
        baseline_model = DecisionTreeRegressor(
            max_depth=5, 
            min_samples_leaf=10, 
            random_state=RANDOM_STATE
        )
        
        logger.info("Training baseline DecisionTree model...")
        rmse, mae, r2, _ = evaluate(baseline_model, X.loc[train_idx], y.loc[train_idx], 
                                   X.loc[test_idx], y.loc[test_idx])
        
        rows.append({"Model": "Baseline_DecisionTree", "RMSE": rmse, "MAE": mae, "R2": r2})
        best = {"Name": "Baseline_DecisionTree", "Model": baseline_model, "RMSE": rmse, "MAE": mae, "R2": r2}
        
        logger.info(f"Baseline model successfully trained: RMSE={rmse:.3f}, MAE={mae:.3f}, R²={r2:.3f}")
        print("WARNING: All advanced models failed - using baseline DecisionTree model")
        
    except Exception as baseline_error:
        logger.error(f"Even baseline model failed: {baseline_error}")
        print("\nCRITICAL ERROR: All models (including baseline) failed to train!")
        print("Possible issues:")
        print("  • Data quality problems (too many missing values)")
        print("  • Insufficient training data")
        print("  • Feature engineering errors")
        print("  • Memory/resource constraints")
        print("\nCheck the logs for detailed error information.")
        raise RuntimeError("All models failed to train. Check data quality and parameters.")

if not rows:  # Final check after baseline attempt
    raise RuntimeError("No models successfully trained.")

results_df = pd.DataFrame(rows).sort_values(["R2","RMSE"], ascending=[False, True]).reset_index(drop=True)
print("\nRegression Model Results:")
print(results_df)
print(f"\nBest Model: {best['Name']} (R²={best['R2']:.3f})")

if model_errors:
    print(f"\nModel Training Errors:")
    for model, error in model_errors.items():
        print(f"  • {model}: {error}")
    
    # Provide recommendations based on common error patterns
    error_messages = " ".join(model_errors.values()).lower()
    if "memory" in error_messages or "memoryerror" in error_messages:
        print("\nRecommendation: Consider reducing CHUNK_SIZE or MAX_MEMORY_GB in configuration")
    if "singular matrix" in error_messages or "linearly dependent" in error_messages:
        print("Recommendation: Check for highly correlated features or constant columns")
    if "sample" in error_messages:
        print("Recommendation: Increase training data or adjust train/test split ratio")

# --- Additional Models: Dryness Classifier & t+1 Forecaster ---

# Dryness Classifier (using configurable threshold from config)
try:
    logger.info(f"Training dryness classifier (threshold: {DRY_THRESHOLD}%)")
    
    y_dry = (y < DRY_THRESHOLD).astype(int)
    dry_ratio = y_dry.mean()
    logger.info(f"Dryness ratio: {dry_ratio:.1%} of samples below {DRY_THRESHOLD}%")
    
    if dry_ratio == 0 or dry_ratio == 1:
        logger.warning(f"Extreme class imbalance: {dry_ratio:.1%} positive class")

    classifiers = {
        "RandomForest": RandomForestClassifier(
            n_estimators=CLF_N_ESTIMATORS, 
            max_depth=CLF_MAX_DEPTH, 
            random_state=RANDOM_STATE, 
            n_jobs=-1,
            class_weight='balanced',  # Handle class imbalance
            oob_score=True
        ),
        "GradientBoosting": GradientBoostingClassifier(
            random_state=RANDOM_STATE,
            validation_fraction=0.1,
            n_iter_no_change=10
        ),
    }

    clf_rows = []
    best_clf = None
    clf_errors = {}

    for name, clf in classifiers.items():
        try:
            clf.fit(X.loc[train_idx], y_dry.loc[train_idx])
            proba = clf.predict_proba(X.loc[test_idx])[:, 1]
            pred = (proba >= 0.5).astype(int)
            
            acc = accuracy_score(y_dry.loc[test_idx], pred)
            f1 = f1_score(y_dry.loc[test_idx], pred, zero_division=0)
            
            try:
                auc = roc_auc_score(y_dry.loc[test_idx], proba)
            except ValueError as e:
                logger.warning(f"AUC calculation failed for {name}: {e}")
                auc = float('nan')
            
            clf_rows.append({"Model": name, "Accuracy": acc, "F1": f1, "AUC": auc})
            
            # Track best classifier
            score = auc if not np.isnan(auc) else f1
            best_score = best_clf.get("AUC" if not np.isnan(auc) else "F1", -1) if best_clf else -1
            
            if best_clf is None or score > best_score:
                best_clf = {"Name": name, "Model": clf, "Accuracy": acc, "F1": f1, "AUC": auc}
            
            logger.info(f"{name} Classifier: Acc={acc:.3f}, F1={f1:.3f}, AUC={auc:.3f}")
            
        except Exception as e:
            error_msg = f"Failed to train {name} classifier: {e}"
            logger.error(error_msg)
            clf_errors[name] = str(e)

    if clf_rows:
        clf_results_df = pd.DataFrame(clf_rows).sort_values(["AUC", "F1", "Accuracy"], 
                                                            ascending=[False, False, False]).reset_index(drop=True)
        print(f"\nDryness Classifier Results (threshold: {DRY_THRESHOLD}%):")
        print(clf_results_df)
        print(f"Best Classifier: {best_clf['Name']}")
    else:
        logger.error("All classifiers failed to train")

except Exception as e:
    logger.error(f"Dryness classification failed: {e}")

# t+1 Moisture Forecaster
try:
    logger.info("Training t+1 forecaster...")
    
    df_base = features.copy()
    site_cols = [c for c in df_base.columns if c.startswith("site_id_")]
    if site_cols:
        df_base["site"] = df_base[site_cols].idxmax(axis=1).str.replace("site_id_", "", regex=False)
    else:
        df_base["site"] = "site"

    df_base["soil_moisture_pct"] = raw.loc[df_base.index, "soil_moisture_pct"]

    def shift_future_target(group):
        group = group.sort_values("timestamp").copy()
        group["y_t1"] = group["soil_moisture_pct"].shift(-1)
        return group

    df_t1 = df_base.groupby("site", group_keys=False).apply(shift_future_target)

    X_t1 = df_t1[feat_cols].astype(float)
    y_t1 = pd.to_numeric(df_t1["y_t1"], errors="coerce")
    mask_t1 = X_t1.notna().all(axis=1) & y_t1.notna()
    X_t1, y_t1 = X_t1[mask_t1], y_t1[mask_t1]

    if len(X_t1) < 50:
        raise ValueError(f"Insufficient data for t+1 forecasting: {len(X_t1)} samples")

    # Use filtered data for t+1 split
    features_t1_filtered = df_t1.loc[mask_t1]
    train_idx_t1, test_idx_t1 = time_aware_split_indices(features_t1_filtered)
    
    # Filter indices to ensure they exist in X_t1
    train_idx_t1 = [idx for idx in train_idx_t1 if idx in X_t1.index]
    test_idx_t1 = [idx for idx in test_idx_t1 if idx in X_t1.index]

    # Train t+1 forecaster
    forecaster = GradientBoostingRegressor(
        random_state=RANDOM_STATE,
        validation_fraction=0.1,
        n_iter_no_change=10
    )
    
    validate_model_inputs(X_t1.loc[train_idx_t1], y_t1.loc[train_idx_t1])
    forecaster.fit(X_t1.loc[train_idx_t1], y_t1.loc[train_idx_t1])

    pred_t1 = np.clip(forecaster.predict(X_t1.loc[test_idx_t1]), 0, 100)
    
    # Fix for older scikit-learn versions
    mse_t1 = mean_squared_error(y_t1.loc[test_idx_t1], pred_t1)
    rmse_t1 = np.sqrt(mse_t1)  # Calculate RMSE manually
    mae_t1 = mean_absolute_error(y_t1.loc[test_idx_t1], pred_t1)
    r2_t1 = r2_score(y_t1.loc[test_idx_t1], pred_t1)

    logger.info(f"t+1 Forecaster: RMSE={rmse_t1:.3f}, MAE={mae_t1:.3f}, R²={r2_t1:.3f}")
    print(f"\nt+1 Forecaster (Gradient Boosting) — RMSE: {rmse_t1:.3f}, MAE: {mae_t1:.3f}, R²: {r2_t1:.3f}")

except Exception as e:
    logger.error(f"t+1 forecasting failed: {e}")
    raise

In [ ]:
# Save model results as CSV
results_df.to_csv("siams_model_results.csv", index=False)
print("Saved model results to: siams_model_results.csv")

In [ ]:
# Predicted vs Actual (test set)
plot_dir = Path("plots")
plot_dir.mkdir(exist_ok=True, parents=True)

def plot_pred_actual(y_true, y_pred, title, fname):
    plt.figure()
    plt.scatter(y_true, y_pred, s=8)
    plt.xlabel("Actual Soil Moisture (%)")
    plt.ylabel("Predicted Soil Moisture (%)")
    plt.title(title)
    plt.tight_layout()
    plt.savefig(plot_dir / fname, dpi=160)
    plt.show()

models_dict = {
    "DecisionTree": DecisionTreeRegressor(random_state=RANDOM_STATE),
    "RandomForest": RandomForestRegressor(
        n_estimators=RF_N_ESTIMATORS, max_depth=RF_MAX_DEPTH, min_samples_leaf=RF_MIN_SAMPLES, 
        random_state=RANDOM_STATE, n_jobs=-1
    ),
    "GradientBoosting": GradientBoostingRegressor(random_state=RANDOM_STATE),
}
if HAVE_XGB:
    models_dict["XGBoost"] = XGBRegressor(
        n_estimators=XGB_N_ESTIMATORS, learning_rate=XGB_LEARNING_RATE, max_depth=XGB_MAX_DEPTH, 
        subsample=0.9, colsample_bytree=0.9, random_state=RANDOM_STATE, n_jobs=-1, 
        reg_lambda=1.0, tree_method="hist"
    )

# Fit models on train and plot
for name, mdl in models_dict.items():
    mdl.fit(X.loc[train_idx], y.loc[train_idx])
    y_pred = np.clip(mdl.predict(X.loc[test_idx]), 0, 100)
    plot_pred_actual(y.loc[test_idx].values, y_pred, f"{name}: Predicted vs Actual", f"pred_vs_actual_{name}.png")

print("Saved plots to:", plot_dir)

In [ ]:
# Feature Importance (tree-based models)
def plot_importances(model, feature_names, title, fname):
    if not hasattr(model, "feature_importances_"):
        return
    imp = model.feature_importances_
    order = np.argsort(imp)[::-1][:20]
    plt.figure(figsize=(8, max(4, len(order)*0.35)))
    plt.barh(range(len(order)), imp[order])
    plt.yticks(range(len(order)), [feature_names[i] for i in order])
    plt.gca().invert_yaxis()
    plt.title(title)
    plt.tight_layout()
    plt.savefig(plot_dir / fname, dpi=160)
    plt.show()

for name, model in models_dict.items():
    plot_importances(model, feat_cols, f"{name} — Feature Importance", f"feat_importance_{name}.png")

print("Saved feature importance plots to:", plot_dir)

## 5. Save Artifacts

In [ ]:
best_model = best["Model"].fit(X.loc[train_idx], y.loc[train_idx])

# Save best regression model
joblib.dump(best_model, "model.joblib")
with open("expected_features.json", "w") as f:
    json.dump(feat_cols, f, indent=2)

# Save dryness classifier if it exists
if "best_clf" in globals() and best_clf and "Model" in best_clf:
    best_clf_model = best_clf["Model"].fit(X.loc[train_idx], (y < DRY_THRESHOLD).astype(int).loc[train_idx])
    joblib.dump(best_clf_model, "dryness_clf.joblib")
    with open("dryness_meta.json", "w") as f:
        json.dump({"threshold": float(DRY_THRESHOLD), "features": list(X.columns)}, f, indent=2)
    print("Saved: dryness_clf.joblib, dryness_meta.json")
else:
    print("Dryness classifier not available — skipped saving.")

# Save t+1 forecaster if it exists
if "forecaster" in globals():
    joblib.dump(forecaster, "model_t1.joblib")
    with open("expected_features_t1.json", "w") as f:
        json.dump(list(X_t1.columns), f, indent=2)
    print("Saved: model_t1.joblib, expected_features_t1.json")
else:
    print("t+1 forecaster not available — skipped saving.")

# Save comprehensive context metadata for the Streamlit app's LLM integration
context_meta = {
    # Core system information
    "known_sites": KNOWN_SITES,
    "dry_threshold_default": float(DRY_THRESHOLD),
    
    # Feature information
    "feature_columns": feat_cols,
    "health_metrics": {
        "vpd_kpa": "Vapor Pressure Deficit (kPa) - plant water stress indicator",
        "dew_point_c": "Dew point temperature (°C) - condensation risk",
        "heat_flag": f"Heat stress flag (temperature ≥ {HEAT_STRESS_TEMP}°C)",
        "frost_flag": f"Frost risk flag (temperature ≤ {FROST_RISK_TEMP}°C)",
        "waterlog_flag": f"Waterlogging risk (soil moisture ≥ {WATERLOG_THRESHOLD}%)",
        "dryspell_flag": f"Drought stress flag (soil moisture ≤ {DRYSPELL_THRESHOLD}%)",
        "disease_risk": "Disease risk score (0-100) based on humidity, temperature, and VPD",
        "sensor_issue_flag": "Sensor malfunction indicator (stuck readings detection)"
    },
    
    # Model performance information
    "model_info": {
        "best_model": best['Name'],
        "models_trained": list(models.keys()) if 'models' in globals() else [],
        "training_samples": len(train_idx),
        "test_samples": len(test_idx),
        "best_r2": float(best['R2']),
        "best_rmse": float(best['RMSE']),
        "best_mae": float(best['MAE']),
        "feature_count": len(feat_cols),
        "health_feature_count": len([c for c in HEALTH_COLS_TRAIN if c in feat_cols])
    },
    
    # Dataset information
    "data_info": {
        "total_records": len(raw),
        "processed_records": len(X),
        "data_quality": f"{len(X)/len(raw)*100:.1f}% usable after cleaning",
        "date_range": {
            "start": str(raw["timestamp"].min()),
            "end": str(raw["timestamp"].max())
        },
        "sites_count": len(raw["site_id"].unique()) if "site_id" in raw.columns else 1,
        "sites_list": sorted(raw["site_id"].unique().tolist()) if "site_id" in raw.columns else ["Unknown"]
    },
    
    # Configuration parameters used
    "config_params": {
        "test_fraction": TEST_FRACTION,
        "random_state": RANDOM_STATE,
        "lag_windows": LAG_WINDOWS,
        "rolling_window": ROLLING_WINDOW,
        "vpd_disease_threshold": VPD_DISEASE_THRESHOLD,
        "heat_stress_temp": HEAT_STRESS_TEMP,
        "frost_risk_temp": FROST_RISK_TEMP,
        "waterlog_threshold": WATERLOG_THRESHOLD,
        "dryspell_threshold": DRYSPELL_THRESHOLD
    },
    
    # Health metrics export info
    "health_export": {
        "filename": "siams_health_metrics.csv",
        "columns": available_health_cols,
        "total_records": len(health_export) if 'health_export' in globals() else 0
    },
    
    # Additional models info
    "additional_models": {
        "dryness_classifier": {
            "available": "best_clf" in globals() and best_clf is not None,
            "threshold": float(DRY_THRESHOLD),
            "model_type": best_clf['Name'] if 'best_clf' in globals() and best_clf else None
        },
        "t1_forecaster": {
            "available": "forecaster" in globals(),
            "model_type": "GradientBoosting" if "forecaster" in globals() else None
        }
    },
    
    # Generation metadata
    "pipeline_info": {
        "generated_at": pd.Timestamp.now().isoformat(),
        "pipeline_version": "SIAMS_ML_v2.0_with_health_metrics",
        "timezone": "Africa/Lagos",
        "data_source": MULTISHEET_XLSX or SHEET_CSV_URL or LOCAL_CSV or "Unknown"
    }
}

# Save context metadata
with open("context_meta.json", "w") as f:
    json.dump(context_meta, f, indent=2)

print("Saved: model.joblib, expected_features.json")
print("Saved: context_meta.json (comprehensive system metadata)")
print(f"Context metadata includes {len(context_meta)} main categories:")
for category in context_meta.keys():
    print(f"  • {category}")

logger.info("All artifacts saved successfully with enhanced metadata support")

results_df